In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda, Dense, Input, Layer, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import scipy
from matplotlib import gridspec

In [12]:
from tensorflow.keras.layers import Layer

initializer = tf.keras.initializers.RandomUniform(minval=-5., maxval=5.)

class MyLayer(Layer):

    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, X):
        return X[1] + X[2]*X[0]

In [13]:
#Quick vanilla GAN from https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-1-dimensional-function-from-scratch-in-keras/
 
# define the standalone discriminator model
def define_discriminator(n_inputs=1):
	model = Sequential()
	model.add(Dense(25, activation='relu', input_dim=n_inputs))
	model.add(Dense(25, activation='relu', input_dim=n_inputs))    
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
 
def define_generator(n_outputs=1):
    inputs = Input(shape=(3, ))
    hidden_layer_1 = Dense(50, activation='relu')(inputs[:,1:2])
    hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
    hidden_layer_3 = Dense(2, activation='relu')(hidden_layer_2)
    concatenate_layer = Concatenate()([hidden_layer_3,inputs[:,0:1]])
    output = concatenate_layer[:, 1]+concatenate_layer[:, 0]*concatenate_layer[:, 2]
    model = Model(inputs=inputs,outputs=output)
    return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model
 
# generate n real samples with class labels
def generate_real_samples(n):
	x = np.random.normal(0.5,1,n)
	y = ones((n, 1))
	b = (np.random.rand(n)*10)-5
	c = (np.random.rand(n)*10)-5
	X = np.c_[x,b,c]
	return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(n):
	# generate points in the latent space
	x_input = generate_real_samples(n)
	return x_input[0]
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, n):
	# generate points in latent space
	x_input = generate_latent_points(n)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n, 1))
	return X, y

def generate_fake_samples_with_input(generator, n):
	# generate points in latent space
	x_input = generate_latent_points(n)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n, 1))
	return X, y, x_input
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, n_epochs=10000, n_batch=128, n_eval=2000):
	# determine half the size of one batch, for updating the discriminator
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# prepare real samples
		x_real, y_real = generate_real_samples(half_batch)
		x_real = x_real[:,0]
		# prepare fake examples
		x_fake, y_fake = generate_fake_samples(g_model, half_batch)
		x_fake = x_fake[:,0]
		# update discriminator
		d_model.train_on_batch(x_real, y_real)
		d_model.train_on_batch(x_fake, y_fake)
		# prepare points in latent space as input for the generator
		x_gan = generate_latent_points(n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, y_gan)
		if (i+1) % n_eval == 0:
			print("epoch = ", i)

In [83]:
N = 1
for j in range(N):
    print("j = ", j)
    # create the discriminator
    discriminator = define_discriminator()
    # create the generator
    generator = define_generator()
    # create the gan
    gan_model = define_gan(generator, discriminator)
    # train model
    train(generator, discriminator, gan_model)
    
    

j =  0


ValueError: Input 0 of layer sequential_12 is incompatible with the layer: : expected min_ndim=2, found ndim=1. Full shape received: [None]

Tensor("dense_41/Identity:0", shape=(None, 2), dtype=float32)


Tensor("strided_slice_17:0", shape=(None, 2), dtype=float32)


In [14]:
my_model = define_generator()
my_model.predict([[1, 2, 3]])

array([0.28059047], dtype=float32)

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 3)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_91 (T [(None, 1)]          0           input_34[0][0]                   
__________________________________________________________________________________________________
dense_96 (Dense)                (None, 50)           100         tf_op_layer_strided_slice_91[0][0
__________________________________________________________________________________________________
dense_97 (Dense)                (None, 50)           2550        dense_96[0][0]                   
___________________________________________________________________________________________

In [88]:
discriminator.trainable = False

In [98]:
model = Sequential()

In [16]:
model.add(generator)
model.compile(loss='binary_crossentropy', optimizer='adam')

NameError: name 'model' is not defined

In [100]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_17 (Model)             (None,)                   2752      
Total params: 2,752
Trainable params: 2,752
Non-trainable params: 0
_________________________________________________________________


In [11]:
    inputs = Input(shape=(3, ))
    hidden_layer_1 = Dense(50, activation='relu')(inputs[:,1:2])
    hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
    hidden_layer_3 = Dense(2, activation='relu')(hidden_layer_2)
    concatenate_layer = Concatenate()([hidden_layer_3,inputs[:,0:1]])
    output = concatenate_layer[:, 1]+concatenate_layer[:, 0]*concatenate_layer[:, 2]
    tf.transpose(output)
    #model = Model(inputs=inputs,outputs=output)
    print(tf.shape(output))

Tensor("Shape:0", shape=(1,), dtype=int32)


In [15]:
generator = define_generator()

In [5]:
	n = 10**3
x = np.random.normal(0.5,1,n)
	y = ones((n, 1))
	b = (np.random.rand(n)*10)-5
	c = (np.random.rand(n)*10)-5

In [9]:
print(np.shape(x))
print(np.shape(b))
print(np.shape(c))

(1000,)
(1000,)
(1000,)
